In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from copy import deepcopy


USA13 = [        
    [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],        
    [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],        
    [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],        
    [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],        
    [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],        
    [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],        
    [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],        
    [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],        
    [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],        
    [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],        
    [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],        
    [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],        
    [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],    
]

N_CIDADES = 13
N_CROMOSSOMO = N_CIDADES - 1 # 

NOMES_CIDADES = [
    "0. New York", "1. Los Angeles", "2. Chicago", "3. Minneapolis",    
    "4. Denver", "5. Dallas", "6. Seattle", "7. Boston",    
    "8. San Francisco", "9. St. Louis", "10. Houston",    
    "11. Phoenix", "12. Salt Lake City"
]

def rota_valida(rota_cromossomo):
    """Verifica se o cromossomo (sem a cidade 0) é válido."""
    if len(rota_cromossomo) != N_CROMOSSOMO:
        return False
        
    if len(set(rota_cromossomo)) != N_CROMOSSOMO:
        return False
        
    if not set(rota_cromossomo) == set(range(1, N_CIDADES)):
        return False

    return True

def calcular_distancia_total(rota_cromossomo):
    """Calcula a distância total, incluindo a volta para a cidade 0."""
    if not rota_valida(rota_cromossomo):
        return float('inf') 
        
    distancia_total = 0

    distancia_total += USA13[0][rota_cromossomo[0]]
    
    for i in range(N_CROMOSSOMO - 1):
        cidade_atual = rota_cromossomo[i]
        proxima_cidade = rota_cromossomo[i+1]
        
        distancia_total += USA13[cidade_atual][proxima_cidade]

    cidade_final = rota_cromossomo[-1]    
    distancia_total += USA13[cidade_final][0]    
    
    return distancia_total

def calcular_fitness(rota_cromossomo):
    """Calcula o fitness (1/dist) para maximização."""
    distancia = calcular_distancia_total(rota_cromossomo)
    
    if distancia == float('inf'):
        return 0.0 
        
    if distancia == 0:
        return 0.0 
        
    return 1.0 / distancia

TAM_POPULACAO = 50
N_GERACOES = 400
TAM_TORNEIO = 3
TAXA_CROSSOVER = 0.90
TAXA_MUTACAO = 0.05
N_ELITE = 5
N_SIMULACOES = 30

def criar_individuo():
    cromossomo = list(range(1, N_CROMOSSOMO + 1))
    random.shuffle(cromossomo)
    return cromossomo

def criar_populacao_inicial():
    populacao = []
    for _ in range(TAM_POPULACAO):
        populacao.append(criar_individuo())
    return populacao

def selecao_torneio(populacao, scores):
    competidores_indices = random.sample(range(TAM_POPULACAO), TAM_TORNEIO)
    
    melhor_indice = -1
    melhor_fitness = -1
    
    for indice in competidores_indices:
        if scores[indice] > melhor_fitness:
            melhor_fitness = scores[indice]
            melhor_indice = indice
            
    return populacao[melhor_indice]

def crossover_order_ox(pai1, pai2):

    filho1 = [None] * N_CROMOSSOMO
    filho2 = [None] * N_CROMOSSOMO

    corte1, corte2 = sorted(random.sample(range(N_CROMOSSOMO), 2))

    filho1[corte1:corte2+1] = pai1[corte1:corte2+1]
    filho2[corte1:corte2+1] = pai2[corte1:corte2+1]
    
    ptr_pai2 = 0
    ptr_filho1 = 0
    cidades_no_filho1 = set(filho1[corte1:corte2+1])
    
    while None in filho1:
        if ptr_filho1 == corte1:
            ptr_filho1 = corte2 + 1
        
        cidade_pai2 = pai2[ptr_pai2]
        if cidade_pai2 not in cidades_no_filho1:
            filho1[ptr_filho1] = cidade_pai2
            ptr_filho1 += 1
        ptr_pai2 += 1

    ptr_pai1 = 0
    ptr_filho2 = 0
    cidades_no_filho2 = set(filho2[corte1:corte2+1])

    while None in filho2:
        if ptr_filho2 == corte1:
            ptr_filho2 = corte2 + 1
        
        cidade_pai1 = pai1[ptr_pai1]
        if cidade_pai1 not in cidades_no_filho2:
            filho2[ptr_filho2] = cidade_pai1
            ptr_filho2 += 1
        ptr_pai1 += 1
        
    return filho1, filho2

def mutacao_swap(individuo):
    pos1, pos2 = random.sample(range(N_CROMOSSOMO), 2)
    
    individuo[pos1], individuo[pos2] = individuo[pos2], individuo[pos1]
    
    return individuo


def executar_ag():
    """Executa uma rodada completa do Algoritmo Genético."""
    populacao = criar_populacao_inicial()
    historico_melhor_fitness = []
    
    melhor_rota_global = None
    melhor_fitness_global = -1

    for geracao in range(N_GERACOES):

        scores = [calcular_fitness(ind) for ind in populacao]
        

        indices_ordenados = np.argsort(scores)[::-1]
        if scores[indices_ordenados[0]] > melhor_fitness_global:
            melhor_fitness_global = scores[indices_ordenados[0]]
            melhor_rota_global = populacao[indices_ordenados[0]]
  
        historico_melhor_fitness.append(melhor_fitness_global)
        
        nova_populacao = []

        for i in range(N_ELITE):
            nova_populacao.append(populacao[indices_ordenados[i]])
        
        while len(nova_populacao) < TAM_POPULACAO:
            pai1 = selecao_torneio(populacao, scores)
            pai2 = selecao_torneio(populacao, scores)
            
            if random.random() < TAXA_CROSSOVER:
                filho1, filho2 = crossover_order_ox(pai1, pai2)
            else:
                filho1, filho2 = deepcopy(pai1), deepcopy(pai2) 
            
            if random.random() < TAXA_MUTACAO:
                filho1 = mutacao_swap(filho1)
            if random.random() < TAXA_MUTACAO:
                filho2 = mutacao_swap(filho2)
            
            nova_populacao.append(filho1)
            if len(nova_populacao) < TAM_POPULACAO:
                nova_populacao.append(filho2)
        
        populacao = nova_populacao 

    return melhor_rota_global, melhor_fitness_global, historico_melhor_fitness

if __name__ == "__main__":
    print(f"--- Iniciando Atividade 6: AG para TSP ---")
    print(f"Config: {N_SIMULACOES} simulações, {N_GERACOES} gerações, Pop={TAM_POPULACAO}")
    
    resultados_finais_fitness = []
    historicos_convergencia = []
    melhor_rota_geral = None
    melhor_fitness_geral = -1
    
    for i in tqdm(range(N_SIMULACOES), desc="Executando Simulações"):
        rota, fitness, historico = executar_ag()
        
        resultados_finais_fitness.append(fitness)
        historicos_convergencia.append(historico)
        
        if fitness > melhor_fitness_geral:
            melhor_fitness_geral = fitness
            melhor_rota_geral = rota
    
    print("\n--- Simulação Concluída ---")

    media_fitness = np.mean(resultados_finais_fitness)
    std_fitness = np.std(resultados_finais_fitness)
 
    distancias_finais = [1.0/f for f in resultados_finais_fitness]
    media_distancia = np.mean(distancias_finais)
    std_distancia = np.std(distancias_finais)
    
    print("\n--- Estatísticas dos Resultados Finais (30 simulações) ---")
    print(f"Fitness Médio: {media_fitness:.10f}")
    print(f"Desvio Padrão (Fitness): {std_fitness:.10f}")
    print(f"Distância Média: {media_distancia:.2f} milhas")
    print(f"Desvio Padrão (Distância): {std_distancia:.2f} milhas")
    
    melhor_distancia_geral = 1.0 / melhor_fitness_geral
    print("\n--- Melhor Rota Encontrada (Global) ---")
    print(f"Distância: {melhor_distancia_geral:.2f} milhas")

    rota_completa_str = "0. New York -> "
    nomes_rota = [NOMES_CIDADES[i].split('. ')[1] for i in melhor_rota_geral]
    rota_completa_str += " -> ".join(nomes_rota)
    rota_completa_str += " -> 0. New York"
    print(f"Rota: {rota_completa_str}")

    media_convergencia = np.mean(historicos_convergencia, axis=0)
    
    plt.figure(figsize=(12, 7))
    plt.plot(media_convergencia)
    plt.title('Gráfico de Convergência Médio do AG (Melhor Fitness por Geração)')
    plt.xlabel('Geração')
    plt.ylabel('Melhor Fitness Médio')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

    df_resultados = pd.DataFrame({'Fitness Final': resultados_finais_fitness})
    
    plt.figure(figsize=(8, 6))
    sns.boxplot(y='Fitness Final', data=df_resultados, palette="viridis")
    plt.title(f'Boxplot dos Resultados Finais do AG ({N_SIMULACOES} simulações)')
    plt.ylabel('Fitness Final (1 / Distância)')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

